In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch.autograd import Variable
from torch.nn import Linear, ReLU, Flatten, Sequential, Conv1d, MaxPool1d, Module, LogSoftmax, NLLLoss, Dropout, MSELoss, Softmax
from torch.optim import Adam, SGD
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import flatten

In [2]:
activity_encode = {"WALKING":0,
       "WALKING_DOWNSTAIRS":1,
       "WALKING_UPSTAIRS":2,
       "LAYING":3,
       "SITTING":4,
       "STANDING":5
       }

In [3]:
"""
vvv
train_x = np.array(train)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)
(((5881, 561), (5881,)), ((1471, 561), (1471,)))
^^^

What I did here was basically  split the data after it was loaded, in this case I have to split the data
before we get to this stage
"""
class UCI_Dynamic_Dataset(Dataset):
    """UCI dataset."""

    def __init__(self, csv_file, root_dir, encode=None):

        data = pd.read_csv(csv_file)
        data['Activity'] = data['Activity'].map(encode)
        data, _ = [x for _, x in data.groupby(data['Activity'] > 2)]
        self.data_y = data['Activity'].values
        data = pd.DataFrame(data.drop(['Activity','subject'],axis=1))
        self.data_x = np.array(data)
        self.root_dir = root_dir
        # [batch, channels, features]
        self.data_x = self.data_x.reshape(len(self.data_x), 1, 561)
        self.data_x  = torch.from_numpy(self.data_x)
        self.data_y = self.data_y.astype(int)
        self.data_y = torch.from_numpy(self.data_y)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]

In [4]:
train_data = UCI_Dynamic_Dataset(csv_file='train.csv', root_dir='',encode=activity_encode)
valid_data = UCI_Dynamic_Dataset(csv_file='valid.csv', root_dir='',encode=activity_encode)
test_data = UCI_Dynamic_Dataset(csv_file='test.csv', root_dir='',encode=activity_encode)


In [5]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

# for inputs, labels in trainloader:
#     row = inputs
#     cols = labels
#     break

In [10]:
class DynamicCNN(Module):
    def __init__(self):
        super(DynamicCNN, self).__init__()

        self.conv1 = Conv1d(1, 100, kernel_size=tuple([3]))
        self.pool =  MaxPool1d(kernel_size=tuple([3]))
        self.fc = Linear(18600, 3)
        self.dropout = Dropout(0.5)

    # Defining the forward pass
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #[32, 100, 186]
        x = x.view(-1, 1 * 100 * 186)
        x = self.dropout(F.softmax(self.fc(x), dim=1))
        return x

In [7]:
#NLLL not MSE because
criterion = NLLLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
def train(model, epochs=5):
    step = 10

    optimizer = Adam(model.parameters(), lr=0.0004)
    model.to(device)

    train_losses, valid_losses = [], []
    for e in range(epochs):
        running_loss = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            optimizer.zero_grad()

            log_ps = model(inputs)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        else:
            valid_loss = 0
            accuracy = 0

            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device).float(), labels.to(device).long()
                    log_ps = model(inputs)
                    valid_loss += criterion(log_ps, labels)

                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

            train_losses.append(running_loss/len(train_data))
            valid_losses.append(valid_loss/len(valid_data))

            if (e+1) % step == 0:
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss),
                      "Validation Loss: {:.3f}.. ".format(valid_loss),
                      "Validation Accuracy: {:.3f}".format(accuracy))



In [11]:

model = DynamicCNN()
model.train()
train(model, epochs=50)

Epoch: 10/50..  Training Loss: -25.000..  Validation Loss: -6.062..  Validation Accuracy: 6.375
Epoch: 20/50..  Training Loss: -25.500..  Validation Loss: -6.650..  Validation Accuracy: 7.350
Epoch: 30/50..  Training Loss: -24.375..  Validation Loss: -6.338..  Validation Accuracy: 6.494
Epoch: 40/50..  Training Loss: -25.188..  Validation Loss: -5.688..  Validation Accuracy: 6.981
Epoch: 50/50..  Training Loss: -24.750..  Validation Loss: -6.000..  Validation Accuracy: 6.594


In [12]:
model.eval()
def test(model):
    """
    Tests model over the testing set
    :param model: to be tested
    """
    test_loss = 0
    accuracy = 0

    # Turn off gradients for testing, saves memory and computations
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            log_ps = model(inputs)
            test_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))

    print("Testing Accuracy: {:.3f}".format(accuracy))
test(model)


Testing Accuracy: 13.304
